In [1]:
import os, glob
print("Affiche le chemin absolu vers le dossier courant"+os.getcwd())

Affiche le chemin absolu vers le dossier courantg:\我的云端硬盘\M1\Méthodologie de la Recherche en Informatique\projet final


In [4]:
import pandas as pd
from pandas.core.frame import DataFrame

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
                 #"MNB":MultinomialNB(),

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

### Lecture du jeu de données Tweets:

In [ ]:
 # the return value will be an iterable object of type TextFileReader:
chunks = pd.read_csv("spam.csv",delimiter=",",chunksize=100000) ## 16*chunk
df = pd.concat([chunk for chunk in chunks]) 
liste_tweets = df["text"].values.tolist() #

In [ ]:
df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [ ]:
# compter NB des spam
text = df["text"]
print("Il y a %s spamhams dans le jeu de données."%(len(text)))
# compter les mots
tokenizer = nltk.RegexpTokenizer("([A-Z]\.[A-Z]?\.?[0-9]?|[0-9]+[,.][0-9]+|[cdjls]'|qu'|[\w'-]+|\S)")
l = [len(tokenizer.tokenize(i)) for i in text]
nb_mots = sum(l)
print("Il y a %s mots dans le jeu de données."%(nb_mots))
# compter les instances négatives et positives
post_negatif= df["spam"].value_counts()
print("Il y a %s spams dans le jeu de données."%(post_negatif[0]))
print("Il y a %s hams positifs dans le jeu de données."%(post_negatif[1]))


nb_ph= 0
for i in text:
    longueur_text = len(sent_tokenize(i))
    nb_ph += longueur_text
print("Il y a %s phrases dans l'ensemble du jeu de données."%(nb_ph))

nb_carac = 0
for ii in text:
    longueur_carac = len(i)
    nb_carac += longueur_carac
print("Il y a %s caractères dans le jeu de données."%(nb_carac))

Il y a 5728 spamhams dans le jeu de données.
Il y a 1878460 mots dans le jeu de données.
Il y a 4360 spams dans le jeu de données.
Il y a 1368 hams positifs dans le jeu de données.
Il y a 91012 phrases dans l'ensemble du jeu de données.
Il y a 13351968 caractères dans le jeu de données.


In [ ]:
liste_classifier = [

    ["naive_bayes",MultinomialNB()],
    ["Logistic Regression", LogisticRegression()],
    ["Perceptron", Perceptron(eta0=0.1, random_state=0,max_iter=10000)],
    ["Tree",DecisionTreeClassifier()],
    ["linear_svc", LinearSVC()]
]

liste_Vectorizer = [
    ["stop",TfidfVectorizer(stop_words='english')],
    ["max_f:20000",TfidfVectorizer(max_features = 20000)],
    ["word-ngram:1-2",TfidfVectorizer(ngram_range = (1,2),analyzer ="word")],
    ["word-ngram:1-3",TfidfVectorizer(ngram_range = (1,3),analyzer ="word")],
    ["char-ngram:1-8",TfidfVectorizer(ngram_range=(1,8), analyzer="char")],  # dico => liste  
]

### Vectorisation du jeu de données

In [ ]:
""" 3 variants --> les param de tfidfVectorisation ; 
                   les paroles pretraitées ou pas ; 
                   les classifieurs 
    @param : list_string ==> pretraite ou pas | sortie : une liste triée (nb de vrai positif = bonne classification) avec les prama et les classifieurs choisis"""

def resultat_param_algo(liste_string):
    liste=[]
    for nom_vec, V in liste_Vectorizer: 
        X = V.fit_transform(liste_string)
        y = df['spam']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

        for nom, classifeur in liste_classifier: # on choisit les algorithmes
            classifeur.fit(X_train, y_train)
            y_pred = classifeur.predict(X_test)
            precision = '%.4f' % (precision_score(y_test, y_pred,average='weighted'))
            recall = '%.4f' % (recall_score(y_test, y_pred,average='weighted'))
            accuracy = '%.4f' % (accuracy_score(y_test, y_pred))
            f1_mesure = '%.4f' % (f1_score(y_test, y_pred,average='weighted'))
            liste.append([precision,recall, accuracy,f1_mesure,nom_vec,nom])
    return liste

def affichier_resultat_param_algo(liste_string):      
    # but : transformer la liste de resultatas en dataframe et l'afficher
    liste_data = resultat_param_algo(liste_string)
    df0 = DataFrame(sorted(liste_data,reverse=True))
    df0.rename({0:'precision', 1:'racall',2:'accuracy',3:'f1-mesure',4:'@param',5:'@algorithme'}, axis='columns',inplace=True)
    return df0

In [ ]:
print("Liste de spam : \n")
affichier_resultat_param_algo(liste_tweets) # 2 min 5.4 s

Liste de spam : 



,precision,racall,accuracy,f1-mesure,@param,@algorithme
0,0.9930,0.9930,0.9930,0.9930,max_f:20000,linear_svc
1,0.9930,0.9930,0.9930,0.9930,max_f:20000,Perceptron
2,0.9919,0.9919,0.9919,0.9919,char-ngram:1-8,Perceptron
3,0.9902,0.9901,0.9901,0.9900,word-ngram:1-2,linear_svc
4,0.9901,0.9901,0.9901,0.9901,word-ngram:1-2,Perceptron
5,0.9895,0.9895,0.9895,0.9895,stop,linear_svc
6,0.9884,0.9884,0.9884,0.9883,char-ngram:1-8,linear_svc
7,0.9813,0.9814,0.9814,0.9813,stop,Perceptron
8,0.9810,0.9808,0.9808,0.9806,word-ngram:1-3,Perceptron
9,0.9796,0.9791,0.9791,0.9787,max_f:20000,Logistic Regression
